<a href="https://colab.research.google.com/github/yoonyoung27/prac_class/blob/main/deeplearning_prac1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

* 런타임 유형 : GPU

In [ ]:
!nvidia-smi

* GPU 확인 : Persistance-M

In [ ]:
import tensorflow
tensorflow.keras.__version__

* [Tensorflow](https://www.tensorflow.org/?hl=ko) : ML 모델을 개발하고 학습시키는 데 도움이 되는 핵심 오픈소스 라이브러리

# A first look at a neural network

This notebook contains the code samples found in Chapter 2, Section 1 of [Deep Learning with Python](https://www.manning.com/books/deep-learning-with-python?a_aid=keras&a_bid=76564dff).

**The problem we are trying to solve here is to classify grayscale images of handwritten digits (28 pixels by 28 pixels), into their 10 categories (0 to 9).** It's a first concrete example of a neural network which makes use of the Python library Keras, MNIST dataset.

In [3]:
from tensorflow.keras.datasets import mnist

(train_images, train_labels), (test_images, test_labels) = mnist.load_data()

11493376/11490434 [==============================] - 0s 0us/step


* mnist.load_data() 함수는 '(x_train, y_train), (x_test, y_test)'값을 반환하므로 (train_images, train_labels), (test_images, test_labels) 변수로 반환값을 받는다.
* `train_images` and `train_labels` form the "training set", the data that the model will learn from. The model will then be tested on the 
"test set", `test_images` and `test_labels`. Our images are encoded as Numpy arrays, and the labels are simply an array of digits, ranging 
from 0 to 9. There is a one-to-one correspondence between the images and the labels.